<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
from collections import Counter
import random
import time
import numpy as np
import math

In [69]:
def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    duracion = time.perf_counter() - inicio
    return resultado, duracion

def calcular_TVD(dict_p1, dict_p2):
    """Distancia Total de Variación (TVD) entre dos distribuciones."""
    soporte_total = set(dict_p1) | set(dict_p2)
    return 0.5 * sum(abs(dict_p1.get(k, 0) - dict_p2.get(k, 0)) for k in soporte_total)

def calcular_momentos(dict_fpm):
    """Calcula esperanza, varianza y desviación estándar de una distribución discreta."""
    esperanza = sum(x * p for x, p in dict_fpm.items())
    varianza = sum((x - esperanza)**2 * p for x, p in dict_fpm.items())
    return esperanza, varianza, varianza**0.5

def comparar_distribuciones_df(p_estimada, p_real):
    soporte = list(p_real.keys())
    data = [{
        "Datos": f"P({x})",
        "Real": p_real.get(x, 0),
        "Estimación": p_estimada.get(x, 0),
        "Diferencia": abs(p_real.get(x, 0) - p_estimada.get(x, 0))
    } for x in soporte]

    esp_r, var_r, desv_r = calcular_momentos(p_real)
    esp_e, var_e, desv_e = calcular_momentos(p_estimada)

    resumen = [
        {"Datos": "Esperanza", "Real": esp_r, "Estimación": esp_e, "Diferencia": abs(esp_r - esp_e)},
        {"Datos": "Varianza", "Real": var_r, "Estimación": var_e, "Diferencia": abs(var_r - var_e)},
        {"Datos": "Desv. Est.", "Real": desv_r, "Estimación": desv_e, "Diferencia": abs(desv_r - desv_e)},
    ]
    return pd.concat([pd.DataFrame(data), pd.DataFrame(resumen)], ignore_index=True)

def calificar_generador_de_muestras(generador, dict_p_real, n=100_000):
  soporte = list(dict_p_real.keys())

  muestras, tiempos = zip(*(medir_tiempo(generador) for _ in range(n)))
  frec = Counter(muestras)
  dict_p_estimada = {x: frec.get(x, 0) / n for x in soporte}

  df_comparacion = comparar_distribuciones_df(dict_p_estimada, dict_p_real)
  tvd = calcular_TVD(dict_p_estimada, dict_p_real)
  t_prom = np.mean(tiempos)

  print(df_comparacion.to_string(index=False))
  print()
  print(f"Distancia total de variación: {tvd:.6f}")
  print(f"Tiempo promedio de ejecución: {t_prom} segundos")


In [70]:
DICT_FPM = {i: p for i, p in enumerate([0.15, 0.20, 0.10, 0.35, 0.20])}
SOPORTE = list(DICT_FPM.keys())
print('funcion de probabilidad de masa: ',DICT_FPM)
print('debe dar 1: ',sum(DICT_FPM.values()))

funcion de probabilidad de masa:  {0: 0.15, 1: 0.2, 2: 0.1, 3: 0.35, 4: 0.2}
debe dar 1:  1.0


In [71]:
# I) Describir mediante un pseudocódigo un algoritmo que simule X utilizando el
# método de la transformada inversa y que minimice el número esperado de búsquedas.

def ordenar_dict_fpm(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  F = {}
  suma = 0
  for valor, probabilidad in dict_fpm.items():
    suma += probabilidad
    F[valor] = suma
  return F

def TI(dict_fda):
  u = random.random()
  for v, p in dict_fda.items():
    if u < p:
      return v

print('Sin ordenar:')
dict_fda = calcular_dict_fda(DICT_FPM)
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador_de_muestras(generador_de_muestras, DICT_FPM)
print()
print('Ordenando:')
dict_fda = calcular_dict_fda(ordenar_dict_fpm(DICT_FPM))
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador_de_muestras(generador_de_muestras, DICT_FPM)

Sin ordenar:
     Datos     Real  Estimación  Diferencia
      P(0) 0.150000     0.14790    0.002100
      P(1) 0.200000     0.19866    0.001340
      P(2) 0.100000     0.09995    0.000050
      P(3) 0.350000     0.35181    0.001810
      P(4) 0.200000     0.20168    0.001680
 Esperanza 2.250000     2.26071    0.010710
  Varianza 1.887500     1.88082    0.006680
Desv. Est. 1.373863     1.37143    0.002433

Distancia total de variación: 0.003490
Tiempo promedio de ejecución: 1.41451272021186e-06 segundos

Ordenando:
     Datos     Real  Estimación  Diferencia
      P(0) 0.150000    0.149660    0.000340
      P(1) 0.200000    0.199880    0.000120
      P(2) 0.100000    0.098100    0.001900
      P(3) 0.350000    0.353080    0.003080
      P(4) 0.200000    0.199280    0.000720
 Esperanza 2.250000    2.252440    0.002440
  Varianza 1.887500    1.884994    0.002506
Desv. Est. 1.373863    1.372951    0.000912

Distancia total de variación: 0.003080
Tiempo promedio de ejecución: 1.31586789977

In [72]:
# II) Describir mediante un pseudocódigo un algoritmo que simule X utilizando el método de aceptación y
# rechazo con una variable soporte Y con distribución binomial B(4,0.45).
N_BIN = 4
P_BIN = 0.45

def crear_dict_fpm(soporte, p):
  return {v: p(v) for v in soporte}

def fpm_binomial(k, n=N_BIN, p=P_BIN):
    if not 0 <= k <= n:
        return 0.0
    coef = math.comb(n, k)
    prob = coef * (p ** k) * ((1 - p) ** (n - k))
    return prob

dict_fpm_binomial = crear_dict_fpm(list(range(N_BIN + 1)), fpm_binomial)
generador_de_muestras_binomial = lambda: TI(calcular_dict_fda(dict_fpm_binomial))

print(DICT_FPM)
print(dict_fpm_binomial)

{0: 0.15, 1: 0.2, 2: 0.1, 3: 0.35, 4: 0.2}
{0: 0.09150625000000003, 1: 0.2994750000000001, 2: 0.3675375000000001, 3: 0.20047500000000004, 4: 0.04100625}


In [74]:
C_posibles = [DICT_FPM[i] / dict_fpm_binomial[i] for i in range(0,len(DICT_FPM))]
C = max(C_posibles)

print(f'C = max({C_posibles}) =',C)

C = max([1.6392322928761691, 0.6678353785791801, 0.2720810801618882, 1.745853597705449, 4.877305288827923]) = 4.877305288827923


In [76]:
def generador_de_muestras_AyR_bin():
  while True:
    y = generador_de_muestras_binomial()
    qy = dict_fpm_binomial[y]
    py = DICT_FPM[y]
    if random.random() < py/(qy*C):
      return y

calificar_generador_de_muestras(generador_de_muestras_AyR_bin, DICT_FPM)

     Datos     Real  Estimación  Diferencia
      P(0) 0.150000    0.149780    0.000220
      P(1) 0.200000    0.198850    0.001150
      P(2) 0.100000    0.101030    0.001030
      P(3) 0.350000    0.349970    0.000030
      P(4) 0.200000    0.200370    0.000370
 Esperanza 2.250000    2.252300    0.002300
  Varianza 1.887500    1.885765    0.001735
Desv. Est. 1.373863    1.373231    0.000632

Distancia total de variación: 0.001400
Tiempo promedio de ejecución: 6.71051957038344e-06 segundos
